In [ ]:
## Openpose in MAS
class SASESP_MAS():
    """
    Class creates ESP project inlcuding OpenPose model
    @author Michael Gorkow
    """
    def __init__(self):
        self.esp_project, self.pub, self.sub = self.esp_buildproject()
        threading.Thread(target=self.sub.ws.run_forever, daemon=True).start()
    def esp_buildproject(self):
        
        #Connect to ESP and create ESP project
        esp = esppy.ESP(hostname='http://localhost:9900')
        esp_project = esp.create_project('test', n_threads=10)
        esp_project.pubsub = 'manual'
        esp_project.add_continuous_query('contquery')

        # Window: Video Capture
        vid_capture = esp.SourceWindow(schema=('id*:int64', 'image:blob'),
        index_type='empty', insert_only=True)
        vid_capture.pubsub = True
        esp_project.windows['w_input'] = vid_capture

        # Window: Encode Binary Image to Text for Python
        func1 = esp.FunctionalWindow(schema=('id*:int64', 'image:blob'),
                                     index_type='empty', 
                                     name='w_func1')
        func1.set_function_context_functions(image='base64EncodeBinary($image)')
        esp_project.windows['w_func1'] = func1

        # Window: Preprocessing in Python
        wpython = esp.PythonHelper(schema=('id*:int64', '_image_scored_:string', 'cmd:string'),
                                   index_type='empty', name='w_python')
        wpython.add_mas_info('module_1', 'score_image', 'w_func1', code_file='/data/notebooks/deep_learning_examples/DJI Tello Drone + OpenPose/openpose_scoring.py')
        esp_project.windows['w_python'] = wpython

        # Window: Encode Image String to Binary
        func2 = esp.FunctionalWindow(schema=('id*:int64', '_image_scored_:blob', 'cmd:string'), 
                                    index_type='empty', 
                                     name='w_func2',
                                     pubsub=True)
        func2.set_function_context_functions(_image_scored_='base64DecodeBinary($_image_scored_)')
        esp_project.windows['w_func2'] = func2

        # Connections
        vid_capture.add_target(func1, role='data')
        func1.add_target(wpython, role='data')
        wpython.add_target(func2, role='data')

        # Load Project
        esp.load_project(esp_project)

        # Publisher: Send Video; Subscriber: Retrieve Results
        pub = vid_capture.create_publisher(blocksize=1, rate=0, pause=0, opcode='insert', format='csv')
        sub = self.esp_sub()
        return esp_project, pub, sub
        
    class esp_sub():
        def __init__(self):
            self.ws = websocket.WebSocketApp("ws://localhost:9900/SASESP/subscribers/test/contquery/w_func2/?format=json&mode=streaming&pagesize=1",
                                             on_message = self.on_message,
                                             on_error = self.on_error,
                                             on_close = self.on_close)
            self.ws.on_open = self.on_open
            self.results = np.zeros((720,960,3), np.uint8)
            self.cmd = 'no_cmd'
                
        def on_message(self, message):
            try:
                data = json.loads(message)
                self.cmd = data['events'][0]['event']['cmd']
                imageBufferBase64 = data['events'][0]['event']['_image_scored_']['_image_scored_']
                nparr = np.frombuffer(base64.b64decode(imageBufferBase64), dtype=np.uint8)
                self.results = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            except Exception as e: 
                print('websocket_error')        

        def on_error(self, error):
            print(error)

        def on_close(self):
            print("### closed ###")

        def on_open(self):
            print('open')
            

    def send(self, frame):
        _, buffer = cv2.imencode('.jpg', frame)
        encoded_string = base64.b64encode(buffer)
        strToSend = 'i, n, ' + str(int(time.time()*100)) + ',' + encoded_string.decode() + ',' + '\n'
        self.pub.send(strToSend)
    
    # Class function to retrieve ESP results
    def get_results(self):
        return self.sub.results
    def get_cmd(self):
        return self.sub.cmd

In [ ]:
class TelloCV(object):
    
    def __init__(self, esp):
        self.prev_flight_data = None
        self.record = False
        self.tracking = False
        self.keydown = False
        self.date_fmt = '%Y-%m-%d_%H%M%S'
        self.speed = 25
        self.out_file = None
        self.out_stream = None
        self.out_name = None
        self.start_time = time.time()
        self.track_cmd = ""
        self.esp = esp
        self.FPS = 18
        self.container = None
        self.video_out = None
        self.time_last_cmd = time.time()
        threading.Thread(target=self.run, daemon=True).start()
        threading.Thread(target=self.display, daemon=True).start()


    def run(self):
        self.drone = tellopy.Tello()
        self.init_drone()
        self.init_controls()
        # container for processing the packets into frames
        try:
            self.container = av.open(self.drone.get_video_stream())
            self.vid_stream = self.container.streams.video[0]
        except:
            exit(0)
        prev = 0
        for packet in self.container.demux((self.vid_stream,)):
            for frame in packet.decode():
                time_elapsed = time.time() - prev
                if time_elapsed > 1./self.FPS:
                    prev = time.time()
                    self.esp.send(np.array(frame.to_image()))
                    try:
                        if self.tracking:
                            self.handle_esp_cmd()
                    except Exception as e:
                        print('cmde: ', e)

    def av_container(self):
        try:
            self.container = av.open(self.drone.get_video_stream())
            return True
        except:
            self.av_container()

    def display(self):
        while True:
            try:
                image = image = self.process_frame()
                cv2.imshow('tello', image)
                _ = cv2.waitKey(1) & 0xFF
            except Exception as e:
                print('display', e)
        

    def init_drone(self):
        """Connect, uneable streaming and subscribe to events"""
        # self.drone.log.set_level(2)
        self.drone.connect()
        self.drone.start_video()
        self.drone.subscribe(self.drone.EVENT_FLIGHT_DATA,
                             self.flight_data_handler)
        self.drone.subscribe(self.drone.EVENT_FILE_RECEIVED,
                             self.handle_flight_received)

    def on_press(self, keyname):
        """handler for keyboard listener"""
        if self.keydown:
            return
        try:
            self.keydown = True
            keyname = str(keyname).strip('\'')
            #print('+' + keyname)
            if keyname == 'Key.esc':
                self.drone.quit()
                exit(0)
            if keyname in self.controls:
                key_handler = self.controls[keyname]
                if isinstance(key_handler, str):
                    getattr(self.drone, key_handler)(self.speed)
                else:
                    key_handler(self.speed)
        except AttributeError:
            print('special key {0} pressed'.format(keyname))

    def on_release(self, keyname):
        """Reset on key up from keyboard listener"""
        self.keydown = False
        keyname = str(keyname).strip('\'')
        #print('-' + keyname)
        if keyname in self.controls:
            key_handler = self.controls[keyname]
            if isinstance(key_handler, str):
                getattr(self.drone, key_handler)(0)
            else:
                key_handler(0)

    def init_controls(self):
        """Define keys and add listener"""
        self.controls = {
            'w': 'forward',
            's': 'backward',
            'a': 'left',
            'd': 'right',
            'Key.space': 'up',
            'Key.shift': 'down',
            'Key.shift_r': 'down',
            'q': 'counter_clockwise',
            'e': 'clockwise',
            'i': lambda speed: self.drone.flip_forward(),
            'k': lambda speed: self.drone.flip_back(),
            'j': lambda speed: self.drone.flip_left(),
            'l': lambda speed: self.drone.flip_right(),
            # arrow keys for fast turns and altitude adjustments
            'Key.left': lambda speed: self.drone.counter_clockwise(speed),
            'Key.right': lambda speed: self.drone.clockwise(speed),
            'Key.up': lambda speed: self.drone.up(speed),
            'Key.down': lambda speed: self.drone.down(speed),
            'Key.tab': lambda speed: self.drone.takeoff(),
            'Key.backspace': lambda speed: self.drone.land(),
            'p': lambda speed: self.palm_land(speed),
            't': lambda speed: self.toggle_tracking(speed),
            'r': lambda speed: self.toggle_recording(speed),
            'z': lambda speed: self.toggle_zoom(speed),
            'Key.enter': lambda speed: self.take_picture(speed)
        }
        self.key_listener = keyboard.Listener(on_press=self.on_press,
                                              on_release=self.on_release)
        self.key_listener.start()
        self.keyboard_ctrl = keyboard.Controller()
    
    def process_frame(self):
        """convert frame to cv2 image and show"""
        #self.esp.send(np.array(frame.to_image()))
        image = self.esp.get_results()
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = self.write_hud(image)
        if self.record:
            if self.video_out != None:
                self.video_out.write(image)
            else:
                print('No VideoWriter created!')
        return image

    def write_hud(self, frame):
        """Draw drone info, tracking and record on frame"""
        try:
            stats = self.prev_flight_data.split('|')
        except:
            stats = []
        stats.append("Tracking:" + str(self.tracking))
        if self.drone.zoom:
            stats.append("VID")
        else:
            stats.append("PIC")
        if self.record:
            diff = int(time.time() - self.start_time)
            mins, secs = divmod(diff, 60)
            stats.append("REC {:02d}:{:02d}".format(mins, secs))

        for idx, stat in enumerate(stats):
            text = stat.lstrip()
            cv2.putText(frame, text, (0, 30 + (idx * 30)),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1.0, (255, 255, 255), lineType=30)
        return frame
    
    def toggle_recording(self, speed):
        """Handle recording keypress, creates output stream and file"""
        if speed == 0:
            return
        self.record = not self.record

        if self.record:
            self.out_name = 'tello-' + datetime.datetime.now().strftime(self.date_fmt) + '.mp4'
            fourcc = cv2.VideoWriter_fourcc(*"avc1")
            self.video_out = cv2.VideoWriter(self.out_name,fourcc, self.FPS, (960,720))
            print("Outputting video to:", self.out_name)

        if not self.record:
            print("Video saved to ", self.out_name)
            self.video_out.release()
            self.video_out = None
                
    def record_vid(self, frame):
        if self.video_out != None:
            self.out.write(frame)
        else:
            print('No VideoWriter created.')

    def take_picture(self, speed):
        """Tell drone to take picture, image sent to file handler"""
        if speed == 0:
            return
        self.drone.take_picture()

    def palm_land(self, speed):
        """Tell drone to land"""
        if speed == 0:
            return
        self.drone.palm_land()

    def toggle_tracking(self, speed):
        """ Handle tracking keypress"""
        if speed == 0:  # handle key up event
            return
        self.tracking = not self.tracking
        print("tracking:", self.tracking)
        return

    def toggle_zoom(self, speed):
        """
        In "video" mode the self.drone sends 1280x720 frames.
        In "photo" mode it sends 2592x1936 (952x720) frames.
        The video will always be centered in the window.
        In photo mode, if we keep the window at 1280x720 that gives us ~160px on
        each side for status information, which is ample.
        Video mode is harder because then we need to abandon the 16:9 display size
        if we want to put the HUD next to the video.
        """
        if speed == 0:
            return
        self.drone.set_video_mode(not self.drone.zoom)

    def flight_data_handler(self, event, sender, data):
        """Listener to flight data from the drone."""
        text = str(data)
        if self.prev_flight_data != text:
            self.prev_flight_data = text

    def handle_flight_received(self, event, sender, data):
        """Create a file in ~/Pictures/ to receive image from the drone"""
        path = '%s/Pictures/tello-%s.jpeg' % (
            os.getenv('HOME'),
            datetime.datetime.now().strftime(self.date_fmt))
        with open(path, 'wb') as out_file:
            out_file.write(data)
        print('Saved photo to %s' % path)                
                
    def handle_esp_cmd(self):
        if time.time() - self.time_last_cmd > 0.1:
            cmdlist = self.esp.get_cmd()
            cmdlist = cmdlist.split(':')
            keys = {
                'Key.up': Key.up,
                'Key.down': Key.down,
                'w': 'w',
                'a': 'a',
                's': 's',
                'd': 'd'
            }
            try:
                released_cmd = list(set(self.prev_cmdlist) - set(cmdlist))
            except Exception as e:
                released_cmd = []
            for cmd in cmdlist:
                if cmd == 'no_cmd':
                    continue
                if cmd == 'right_hand_top':
                    self.drone.flip_left()
                if cmd == 'left_hand_top':
                     self.drone.flip_right()
                if cmd == 'hand_fold':
                    self.drone.land()
                if cmd == 'from_behind':
                    self.drone.flip_back()
                if cmd in keys:
                    cmd = keys[cmd]
                    self.keyboard_ctrl.press(cmd)
            for cmd in released_cmd:
                if cmd in keys:
                    cmd = keys[cmd]
                    self.keyboard_ctrl.release(cmd)
            self.prev_cmdlist = cmdlist
            self.time_last_cmd = time.time()

In [ ]:
## Test functionality with Drone
import cv2
import numpy as np
import time
import datetime
import os
import tellopy
import av
import threading
import base64
from pynput import keyboard
import esppy
import websocket
import json
from pynput.keyboard import Key

# Connect Tello WIFI
connect_counter = 0
success = 1
while success != 0:
    os.system('nmcli device wifi rescan')
    success = os.system('nmcli device wifi connect TELLO-FD9FBA')
    connect_counter += 1
    time.sleep(2)
print('Drone connected.')


e = SASESP_MAS() # Start ESP project
time.sleep(2)
d = TelloCV(e) # Start drone & publish frames to ESP